In [45]:
import torch.nn as nn
class TorchNN(nn.Module):
    # 初始化
    def __init__(self):  # self 指代新创建模型对象
        super().__init__()

        self.linear1 = nn.Linear(4096, 512)
        self.bn1 = nn.BatchNorm1d(512)
        self.linear2 = nn.Linear(512, 512)
        self.bn2 = nn.BatchNorm1d(512)
        self.linear3 = nn.Linear(512, 40)
        self.drop = nn.Dropout(p=0.5)
        self.act = nn.ReLU()

    # forward 前向运算 (nn.Module方法重写)
    def forward(self, input_tensor):
        out = self.linear1(input_tensor)
        out = self.bn1(out)
        out = self.act(out)
        out = self.drop(out)
        out = self.linear2(out)
        out = self.bn2(out)
        out = self.act(out)
        out = self.drop(out)
        final = self.linear3(out) # shape

        return final

In [4]:
# 导入脸谱数据集
from sklearn.datasets import fetch_olivetti_faces

In [5]:
# 查看数据集的形状
olivetti_faces = fetch_olivetti_faces(data_home='./face_data', shuffle=True,download_if_missing=True)
print(olivetti_faces.data.shape)
print(olivetti_faces.target.shape)
print(olivetti_faces.images.shape)

(400, 4096)
(400,)
(400, 64, 64)


In [54]:
import torch
images = torch.tensor(olivetti_faces.data)
targets = torch.tensor(olivetti_faces.target)

dataset = [(img,lbl) for img,lbl in zip(images, targets)]

In [51]:
# 定义超参数
LR = 1e-3
EPOCHS = 10
BATCH_SIZE = 30

In [57]:
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(images, targets, test_size=0.2)
dataset = [(x, y) for x, y in zip(X_train, y_train)]
trainloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
dataset_test  = [(x, y) for x, y in zip(X_test, y_test)]
testloader = DataLoader(dataset_test, batch_size=BATCH_SIZE, shuffle=False)


In [28]:
# 定义模型
model = TorchNN()

In [60]:

# 定义损失函数和优化器
# 损失函数&优化器
loss_fn = nn.CrossEntropyLoss()  # 交叉熵损失函数
# 优化器（模型参数更新）
optimizer = torch.optim.AdamW(model.parameters(), lr=LR)

In [61]:
model.train()
for epoch in range(EPOCHS):
    for out, target in trainloader:
        out = model(out)
        loss = loss_fn(out,target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'epoch:{epoch},loss:{loss.item()}')

epoch:0,loss:0.0011089087929576635
epoch:1,loss:0.0034102301578968763
epoch:2,loss:0.0014083718415349722
epoch:3,loss:0.01115609984844923
epoch:4,loss:0.001128973439335823
epoch:5,loss:0.03192413970828056
epoch:6,loss:0.0028618755750358105
epoch:7,loss:0.0027046343311667442
epoch:8,loss:0.0015108899679034948
epoch:9,loss:0.00048065505689010024


In [62]:
# 测试数据的准确性
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in testloader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print(f'Accuracy: {correct/total*100}%')

Accuracy: 98.75%
